<a href="https://colab.research.google.com/github/DoubekSeth/CS5350HW/blob/main/HW1/CS5350HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter
import queue
import numpy as np

Creating Basic ID3 algorithm

In [2]:
class Node:
  def __init__(self, s, atrs, parent=None, parentVal=None, label=None):
    self.s = s
    self.atrs = atrs
    self.parent = parent
    self.label = label
    self.children = []
    self.parentVal = parentVal

  def addChild(self, node):
    self.children.append(node)

In [50]:
def ID3(s, remaining_attributes, full_attributes, purity_func, parentNode):
  #First, check if all training examples s have same label. First stopping condition
  if(labelSame(s)[0]):
    #print("Same Label", s)
    parentNode.label = labelSame(s)[1]
    return
  #Second, check if exhausted attributes and if so return leaf node with most common label. Second stopping condition
  if(len(remaining_attributes) == 0):
    #print("Exhausted Attributes", s)
    parentNode.label = mostCommonLabel(s, 1)[0][0]
    return
  #If didn't stop, then we can move onto the main part of the algorithm
  #Create root if we don't have one
  if(parentNode==None):
    parentNode = Node(s=s, atrs=remaining_attributes, parent=None)
  #Find attribute best splits S
  bestAtr = findBestSplit(s, remaining_attributes, full_attributes, purity_func)
  parentNode.label = bestAtr
  #print(bestAtr)
  less_remaining_attributes = remaining_attributes[:]
  less_remaining_attributes.remove(bestAtr)
  #Go thru all values for best attribute
  for val in full_attributes[bestAtr]:
    s_v = filterSOnAtrVal(s, bestAtr, val, full_attributes)
    #Check if s_v empty
    if(len(s_v)==0):
      #print("S_v empty", val)
      newNode = Node(s=s, atrs=less_remaining_attributes, parent=parentNode, parentVal=val, label=mostCommonLabel(s, 1)[0][0])
      parentNode.addChild(newNode)
      return newNode
    else:
      newNode = Node(s=s_v, atrs=less_remaining_attributes, parent=parentNode, parentVal=val)
      #print("New node", val)
      parentNode.addChild(newNode)
      ID3(s_v, less_remaining_attributes, full_attributes, purity_func, newNode)
  return parentNode

Does two functions, first returns true if all labels are the same for s. Second returns the label if all labels are same or else returns none

In [4]:
def labelSame(s):
  label = s[0][len(s[0])-1]
  for i in s:
    if(i[len(i)-1] != label):
      return False, None
  return True, label

Another helper function, returns the n most common label in s

In [5]:
def mostCommonLabel(s, n):
  labels = []
  for i in s:
    labels.append(i[len(i)-1])
  #Find most common label out of all the labels
  mostCommon = Counter(labels).most_common(n)
  return mostCommon

This helper finds the attribute that results in the best split

In [6]:
def findBestSplit(s, atrs, full_attributes, purity_func):
  max_info_gain = 0
  max_info_gain_atr = ""
  for atr in atrs:
    info_gained = find_info_gained(s, atr, full_attributes, purity_func)
    if(info_gained > max_info_gain):
      max_info_gain = info_gained
      max_info_gain_atr = atr
  return max_info_gain_atr

This helper gives the information gained from a single attribute

In [7]:
def find_info_gained(s, atr, full_attributes, purity_func):
  atr_ind = list(full_attributes.keys()).index(atr)
  #Create dictionary that contains attribute categories and what labels each attribute category has
  atr_type_count_dict = {}
  for i in s:
    if i[atr_ind] in atr_type_count_dict:
      atr_type_count_dict[i[atr_ind]].append(i[len(i)-1])
    else:
      atr_type_count_dict[i[atr_ind]] = [i[len(i)-1]]
  #Use proportions to run info gain function and find total information gain
  starting_info = purity_func(list(x[len(x)-1] for x in s))
  weighted_sum = 0
  for j in atr_type_count_dict.values():
    weighted_sum += len(j)/len(s)*purity_func(j)
  return starting_info-weighted_sum

Filters S based on an attribute value to get S_v

In [8]:
def filterSOnAtrVal(s, atr, val, full_attributes):
  filtered = []
  atr_ind = list(full_attributes.keys()).index(atr)
  for i in s:
    if(i[atr_ind] == val):
      filtered.append(i)
  return filtered

Below are all purity functions, they simply take in a list of labels and output

In [9]:
def entropy(labels):
  vec_counts = np.array(list(Counter(labels).values()))
  vec_props = vec_counts/np.sum(vec_counts)
  return -np.dot(vec_props, np.log2(vec_props))

Below is the code to unravel a tree and make it presentable

In [48]:
def unravelTree(q1, q2, childQ, level):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  while(not q1.empty()):
    node = q1.get()
    childThere = childQ.get()
    firstChild = True
    print("ParentVal: ", node.parentVal)
    print("Label: ", node.label)
    if(not childThere):
      print("----")
    for child in node.children:
      q2.put(child)
      if(firstChild):
        firstChild = False
      else:
        childQ.put(True)
    if(len(node.children)!= 0):
      childQ.put(False)
  level += 1
  q1 = q2
  q2 = queue.Queue()
  if(q1.empty()):
    return
  unravelTree(q1, q2, childQ, level)

In [52]:
s = [["sunny", "hot", "high", "weak", "-"],
     ["sunny", "hot", "high", "strong", "-"],
     ["overcast", "hot", "high", "weak", "+"],
     ["rainy", "mild", "high", "weak", "+"],
     ["rainy", "cool", "normal", "weak", "+"],
     ["rainy", "cool", "normal", "strong", "-"],
     ["overcast", "cool", "normal", "strong", "+"],
     ["sunny", "mild", "high", "weak", "-"],
     ["sunny", "cool", "normal", "weak", "+"],
     ["rainy", "mild", "normal", "weak", "+"],
     ["sunny", "mild", "normal", "strong", "+"],
     ["overcast", "mild", "high", "strong", "+"],
     ["overcast", "hot", "normal", "weak", "+"],
     ["rainy", "mild", "high", "strong", "-"]]
full_attributes = {"outlook":["sunny", "overcast", "rainy"], "temperature":["hot", "mild", "cool"], "humidity":["high", "normal", "low"], "wind":["strong", "weak"]}
remaining_attributes = ["outlook", "temperature", "humidity", "wind"]


root = ID3(s, remaining_attributes, full_attributes, entropy, None)


q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)
unravelTree(q, q2, childQ, 0)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  None
Label:  outlook
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  sunny
Label:  humidity
ParentVal:  overcast
Label:  +
ParentVal:  rainy
Label:  wind
----
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  high
Label:  -
ParentVal:  normal
Label:  +
ParentVal:  low
Label:  -
----
ParentVal:  strong
Label:  -
ParentVal:  weak
Label:  +
----
